In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import util
from numpy import linalg as LA
import pickle
import random

from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.multiclass import OneVsRestClassifier


In [21]:
df = util.get_processed_data("./data/collections_math.csv", False)
collections = util.get_collections(df)

In [111]:
df.tail(10)

,collection_id,sequence_id,resource_id,title,description,text
27744,ffea9e8f-65e1-4731-bebb-140174f60cda,5,932997a9-c44c-479c-ab9d-ec10fe17fc7f,"Given the polar coordinates of Point V (2,&nbs...","Given the polar coordinates of Point V (2,&nbs...",Given the polar coordinates of Point V nbsp s ...
27745,ffea9e8f-65e1-4731-bebb-140174f60cda,4,d7336856-48c1-4143-95b0-a64e3252d207,"Given the polar coordinates of Point D (4,&nbs...","Given the polar coordinates of Point D (4,&nbs...",Given the polar coordinates of Point D nbsp s ...
27746,ffea9e8f-65e1-4731-bebb-140174f60cda,1,81bf920e-ee87-4b68-9674-3aed717521d6,Given the rectangular coordinates of Point P (...,Given the rectangular coordinates of Point P (...,Given the rectangular coordinates of Point P G...
27747,ffea9e8f-65e1-4731-bebb-140174f60cda,3,c3a6e776-f263-47df-9fd2-91087a183539,Given the rectangular coordinates of Point L (...,Given the rectangular coordinates of Point L (...,Given the rectangular coordinates of Point L G...
27748,ffea9e8f-65e1-4731-bebb-140174f60cda,2,d98c1c9e-4e76-4d3d-b3ff-7577415ada69,Given the rectangular coordinates of Point M (...,Given the rectangular coordinates of Point M (...,Given the rectangular coordinates of Point M G...
27749,fff795b2-263c-48cb-8f93-74693e8c9c58,5,87f88be6-843f-11e3-a253-12313f070480,Simplify 5^4/5^1.,Simplify 5^4/5^1.,Simplify Simplify
27750,fff795b2-263c-48cb-8f93-74693e8c9c58,1,87f88984-843f-11e3-a253-12313f070480,Chapter 8: Exponential Functions,This textbook chapter on exponential functions...,Chapter Exponential Functions This textbook ch...
27751,fff795b2-263c-48cb-8f93-74693e8c9c58,3,87f88830-843f-11e3-a253-12313f070480,Division with exponents,"IXL presents Division with exponents, an educa...",Division with exponents IXL presents Division ...
27752,fff795b2-263c-48cb-8f93-74693e8c9c58,2,87f886f0-843f-11e3-a253-12313f070480,Exponent Properties Involving Quotients,This video from Khan Academy explain exponent ...,Exponent Properties Involving Quotients This v...
27753,fff795b2-263c-48cb-8f93-74693e8c9c58,4,87f88aba-843f-11e3-a253-12313f070480,Simplify (5^3*5^5)/5^9.,Simplify (5^3*5^5)/5^9.,Simplify Simplify


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27754 entries, 0 to 27753
Data columns (total 6 columns):
collection_id    27754 non-null object
sequence_id      27754 non-null int64
resource_id      27754 non-null object
title            27754 non-null object
description      27754 non-null object
text             27754 non-null object
dtypes: int64(1), object(5)
memory usage: 1.3+ MB


In [62]:
#Removing words with length 1 since tfidf does not recognize them
for col in collections:
    lrs = []
    for i,text in enumerate(col["texts"]):
        new_text = ""
        for word in text.split():
            if len(word)>2:
                new_text+=word+" "
        if new_text.strip()!="":
            lrs.append(new_text.strip().lower())
    col["texts"] = lrs

In [63]:
d2v= Doc2Vec.load("doc2vec_100dim.model")

def document_embeddings(doc):
    test_data = word_tokenize(doc.lower())
    return d2v.infer_vector(test_data)        


word2vec = Word2Vec.load('Word2Vec_100dim.bin')

def Word2doc(doc):
    words=doc.split()
    emb=np.zeros(100)
    for word in words:
        if(word in word2vec.wv.vocab):
            emb = np.add(emb,word2vec[word])
    if(len(doc)!=0) :       
        return emb/len(words)
    else:
        return(np.zeros(100))

In [64]:
print(document_embeddings(""))
print("-----------------------------")
print(Word2doc(""))

[ 0.00048814  0.00215189  0.00102763  0.00044883 -0.00076345  0.00145894
 -0.00062413  0.00391773  0.00463663 -0.00116558  0.00291725  0.00028895
  0.00068045  0.00425597 -0.00428964 -0.00412871 -0.00479782  0.0033262
  0.00278157  0.00370012  0.00478618  0.00299159 -0.00038521  0.00280529
 -0.00381726  0.00139921 -0.00356647  0.00444669  0.00021848 -0.00085338
 -0.00235444  0.00274234 -0.0004385   0.00068434 -0.0048121   0.00117635
  0.00112096  0.00116934  0.00443748  0.0018182  -0.00140492 -0.00062968
  0.00197631 -0.00439775  0.00166767  0.00170638 -0.00289617 -0.00371074
 -0.00184572 -0.00136289  0.00070197 -0.00061398  0.00488374 -0.00397955
 -0.00291123 -0.0033869   0.00153108 -0.00246708 -0.00033689 -0.00255574
 -0.0034103  -0.00389625  0.0015633  -0.00361817 -0.00303418 -0.00131275
  0.00320993 -0.00402899  0.00337945 -0.00403902  0.00476459 -0.00031349
  0.00476761  0.00104846  0.00239264 -0.00460812 -0.00217193 -0.00379803
 -0.0020386  -0.00381272 -0.00182017 -0.00085737 -0.

In [65]:
pickle_in = open("./data/svm_model_cosine.sav","rb")
svm = pickle.load(pickle_in)


In [66]:
def cosin(v1,v2):
    if(LA.norm(v1)!=0 and LA.norm(v2)!=0):
        return (np.dot(np.array(v1),np.array(v2))/(LA.norm(v1) * LA.norm(v2)))
    else:
        return 1  


In [73]:
def text_similarity_svm(text1, text2):
    
    dv1 = document_embeddings(text1)
    dv2 = document_embeddings(text2)
    
    wd1 = Word2doc(text1)
    wd2 = Word2doc(text1)
    
    ds = cosin(dv1,dv2)
    ws = cosin(wd1,wd2)

    data=[]

#     for i in range(0,100):
#         data.append(dv1[i])
    
#     for i in range(0,100):
#         data.append(dv2[i])
    
#     for i in range(0,100):
#         data.append(wd1[i])
    
#     for i in range(0,100):
#         data.append(wd2[i])
    
    data.append(ds)
    data.append(ws)
    
    similarity = svm.predict_proba(np.array(data).reshape(1,2))[0][1]

    
    return similarity

In [74]:
def get_mean_score_pair(data, score_function):
    score_sum = 0.0
    for lr1, lr2 in data:
        score_sum += score_function(lr1, lr2)
    return score_sum/len(data)

In [75]:
def get_next_lr(given_lr, lr_list, target_score, score_function):
    next_lr = ""
    score_diff = 0
    for lr in lr_list:
        score = score_function(given_lr, lr)
        if (score - target_score) > score_diff:
            score_diff = (score - target_score)
            next_lr = lr
    return next_lr

In [76]:
def create_collection(collection_size, start_lr, lr_list, target_score, score_function):
    collection = [start_lr]
    lr = start_lr
    if(start_lr in lr_list): 
        lr_list.remove(start_lr)
    for _ in range(collection_size - 1):
        next_lr = get_next_lr(lr, lr_list, target_score, score_function)
        collection.append(next_lr)
        lr = next_lr
        if(next_lr in lr_list):
            lr_list.remove(next_lr)
    return collection

In [77]:
lr_pairs = []

for col in collections:
    for i in range(len(col["texts"]) -1):
        lr1 = col["texts"][i]
        lr2 = col["texts"][i+1]
        lr_pairs.append((lr1, lr2))

In [78]:
mean = get_mean_score_pair(lr_pairs, text_similarity_svm)

/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [79]:
lr_lists = []

for col in collections:
    for lr in col["texts"]:
        if(len(lr)>0):
            lr_lists.append(lr)

In [80]:
lr_lists=list(set(lr_lists))

In [81]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)

which these sums equal which these sums equal frac frac choose all answers that apply


In [88]:
new_collection = create_collection(8, start_lr, lr_lists, 0.5, text_similarity_svm)

/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [89]:
# print(new_collection)

for i in new_collection:
    print(i)
    print("----------------------------------------------------------------")

learn comparing areas plots land find area two rectangles solve word problem
----------------------------------------------------------------
lesson concept development draw the labeled tape diagram the board and give students the context have them write story problem based the tape diagram
----------------------------------------------------------------
solve draw tape diagram solve draw tape diagram number bond add subtract tens write the newnumber sentence
----------------------------------------------------------------
write statement express each conversion write statement express each conversion the first one done for you the screen measures inches convert inches feet inches inch feet feet feetthe screen measures inches feet jug syrup holds cups convert cups pints pints the length the diving board centimeters what its length meters meters the capacity container milliliters convert this liters liters truck weighs grams convertthe truck weight kilograms kilograms the distance was m

In [90]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection1 = create_collection(8, start_lr, lr_lists, 0.6, text_similarity_svm)

finding percent change practice practice finding percent change with answers kuta


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [91]:
for i in new_collection1:
    print(i)
    print("----------------------------------------------------------------")

finding percent change practice practice finding percent change with answers kuta
----------------------------------------------------------------
complete each pattern complete each pattern
----------------------------------------------------------------
splitting the linear term introduce the following strategy splitting the linear term into two terms and regrouping and apply the second problem above
----------------------------------------------------------------
lesson ruling out chance explain writing what you learned about randomly divided groups from the last lesson
----------------------------------------------------------------
multiply whole numbers times fractions did you get the same answer using both methods
----------------------------------------------------------------
lesson describing the center distribution using the median suppose chain restaurant restaurant advertises that typical number french fries large bag
-------------------------------------------------------

In [92]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection2 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)

what fraction full turn what fraction full turn


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [93]:
for i in new_collection2:
    print(i)
    print("----------------------------------------------------------------")

what fraction full turn what fraction full turn
----------------------------------------------------------------
pythagorean theorem side units long and the adjacent square has area units
----------------------------------------------------------------
sequence shown below what function that defines the nth term the sequence sequence shown below what function that defines the nth term the sequence
----------------------------------------------------------------
take out the tens decomposing whole numbers into tens and ones foundational for today lesson
----------------------------------------------------------------
consider the sequence that follows consider the sequence that follows plus pattern write formula for the sequence using both the notation and the notation does the formula generate the same sequence why might some people prefer thisformula graph the terms the sequence ordered pairs the coordinate plane what you noticeabout the graph
-----------------------------------------

In [94]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection3 = create_collection(8, start_lr, lr_lists, 0.8, text_similarity_svm)

simplify the expression using one more properti simplify the expression using one more properties addition and multiplication left frac right left right left frac right left right


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [95]:
for i in new_collection3:
    print(i)
    print("----------------------------------------------------------------")

simplify the expression using one more properti simplify the expression using one more properties addition and multiplication left frac right left right left frac right left right
----------------------------------------------------------------
suppose the times that five students studied suppose the times that five students studied for test are follows michelle said that the mad for this data set hours because the dot plot balanced around without doing any calculations you agree with michelle why why not
----------------------------------------------------------------
eight each two varieties tomato plants eight each two varieties tomato plants loveem and wonderful are grown under the same conditions
----------------------------------------------------------------
multiply
----------------------------------------------------------------
evaluate what
----------------------------------------------------------------
lesson reason concretely and pictorially using place value understandin

In [102]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection4 = create_collection(8, "solving systems equations using linear combinations addition method learn solve systems equations using the linear combination method"
                                    , lr_lists, 0.8, text_similarity_svm)

video formula for geometric series khan practice this lesson yourself khanacademy org right now https www khanacademy org math precalculus seq induction geometric sequence series geometric series utm source utm medium desc utm campaign precalculus watch the next lesson https www khanacademy org math precalculus seq indu


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [103]:
for i in new_collection4:
    print(i)
    print("----------------------------------------------------------------")

solving systems equations using linear combinations addition method learn solve systems equations using the linear combination method
----------------------------------------------------------------
jean luc jogged around the lake jean luc jogged around the lake frac frac hour william jogged the same distance frac frac hour how muchlonger did jean luc take than william hours
----------------------------------------------------------------
chris nbsp playing with die chris cups his chris nbsp playing with die chris cups his hand hopes for six and rolls six chris convinced that lucky taylor comes along drops another die the table and tells chris again with both them can easily with two with one says chris what the probability rolling six the probability what the probability rolling double sixes the probability
----------------------------------------------------------------
write statement express each conversion write statement express each conversion the first one done for you convert 

In [96]:
len(lr_lists)

17453

In [101]:
for i in lr_lists:
    if("linear" in i):
        print (i)

lesson lesson summary when the relationship between two numerical variables and linear straight line can used describes the relationship
linear regression model remember that module used graphing display calculator gdc find linear regression model
lesson modeling context from data this real life descriptive modeling lesson about creating different types functions based data including linear quadratic and exponential
powerpoint linear quadratic exponential powerpoint linear quadratic exponential
some ordered pairs for linear function are given the table below which the following equations was used generate the table above some ordered pairs for linear function are given the table below which the following equations was used generate the table above
number solutions equations one variable linear equations practice telling whether equation has one zero infinite solutions for example how many solutions does the equation
linear inequalities linear inequality describes area the coordinate pl

In [104]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")


leah years older than sue and john years older than leah and the total their ages then how old sue leah years older than sue and john years older than leah and the total their ages then how old sue


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


leah years older than sue and john years older than leah and the total their ages then how old sue leah years older than sue and john years older than leah and the total their ages then how old sue
----------------------------------------------------------------
lesson use metric measurement model the decomposition one whole into tenths
----------------------------------------------------------------
compound sample spaces explore the notion sample space see sample space represented tree diagram table and list
----------------------------------------------------------------
video percent word problems khan academy presents percent word problems educational video resource math
----------------------------------------------------------------
application problem aneisha setting play space for her new puppy
----------------------------------------------------------------
constructing the number line the purpose this exercise let students construct the number line positive and negative numb

In [105]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")


video inequalities with word problems writing inequalities using information from the word problems


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


video inequalities with word problems writing inequalities using information from the word problems
----------------------------------------------------------------
the absolute value any number the absolute value any number will always positive number positive numbers will always have higher absolute value than negative numbers
----------------------------------------------------------------
another way make the same total value another way make the same total value
----------------------------------------------------------------
rates proportional relationships variety questions about rates and proportional relationships involving graphs tables and equations
----------------------------------------------------------------
the long division algorithm for polynomial division when solving the problem example sure record the polynomial division problem next the arithmetic problem already the board
----------------------------------------------------------------
graph the equation the coo

In [106]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")


change mixed numbers improper fractions this fluency activity reviews module concepts


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


change mixed numbers improper fractions this fluency activity reviews module concepts
----------------------------------------------------------------
lesson solution sets simultaneous circle all the ordered pairs that are solutions the inequality below
----------------------------------------------------------------
graphing lines sketch the graph each line
----------------------------------------------------------------
draw show how children can equally share draw show how children can equally share cookies write equation and express your answer asa fraction
----------------------------------------------------------------
lesson lesson summary the range the tangent function all real numbers
----------------------------------------------------------------
video interpreting transformation explain interpreting the graph transformation with video tutorials and quizzes using our many ways approach from multiple teachers this lesson shows how determine how function was transformed observ

In [107]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")


find how many times greater the surface area the surface area jupiter and the surface area earth without using acalculator but using the table logarithms find how many times greater the surface area jupiter than the surfacearea earth


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


find how many times greater the surface area the surface area jupiter and the surface area earth without using acalculator but using the table logarithms find how many times greater the surface area jupiter than the surfacearea earth
----------------------------------------------------------------
effects bracing adolescents the role bracing patients with adolescent idiopathic scoliosis who are risk for curve progression and eventual surgery controversial
----------------------------------------------------------------
david was when marieka was born david was when marieka was born complete the table
----------------------------------------------------------------
what the weight one chair pounds solve using tape diagrams table and chairs weigh together the table weighs what the weight onechair pounds
----------------------------------------------------------------
evaluate evaluate uwhen and frac frac
----------------------------------------------------------------
david the groundske

In [108]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")

    

eyeballing the line best fit given random assortment points draw line best fit through them


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


eyeballing the line best fit given random assortment points draw line best fit through them
----------------------------------------------------------------
how many ounces does each kitten weigh solve liza cat had six kittens when liza and her brother weighed all the kittens together they weighed pounds ounces since all the kittens are about the same size about how many ounces does each kittenweigh
----------------------------------------------------------------
striped marlin can swim rate miles striped marlin can swim rate miles per hour this faster slower rate than sailfish which takes minutes swim miles
----------------------------------------------------------------
find the selling price each item company offers off everything the store but there sales tax costumer wantssome items that have retail value how much will the costumer pay
----------------------------------------------------------------
how much money does krista need give malory use the rdw process solve write your a

In [109]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")


what the length the shaded part what the length the shadedpart the meter stick incentimeters what fraction meter centimeters fraction form express the lengthof the shaded portion the meterstick decimal form express the length the shaded portion the meter stick what fraction meter centimeters


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


what the length the shaded part what the length the shadedpart the meter stick incentimeters what fraction meter centimeters fraction form express the lengthof the shaded portion the meterstick decimal form express the length the shaded portion the meter stick what fraction meter centimeters
----------------------------------------------------------------
absolute value and opposite integers fun math practice improve your skills with this interactive resource from ixl immersive learning experience that provides comprehensive standards aligned content for math language arts science and social studies
----------------------------------------------------------------
barb used her scissors cut out coupon barb used her scissors cut out coupon from the newspaper how many quarter turns does she need turn the paper order stay the lines
----------------------------------------------------------------
completion the exercise graph the number and its opposite the number line graph the number and 

In [110]:
start_lr = lr_lists[random.randint(0,len(lr_lists))]
print(start_lr)
new_collection6 = create_collection(8, start_lr, lr_lists, 0.7, text_similarity_svm)
for i in new_collection6:
    print(i)
    print("----------------------------------------------------------------")


partition and shade the following shapes indicated split the pizza below that maria paul jose and mark each have equal share


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


partition and shade the following shapes indicated split the pizza below that maria paul jose and mark each have equal share
----------------------------------------------------------------
use this example show students use this example show students how rounding addends minuends and subtrahends can help predict reasonable answers
----------------------------------------------------------------
point slope form write equation from graph fun math practice improve your skills with this interactive resource from ixl immersive learning experience that provides comprehensive standards aligned content for math language arts science and social studies
----------------------------------------------------------------
answer the following question evaluate the expression nbsp
----------------------------------------------------------------
lesson student debrief use measurement tools convert mixed number measurements smaller units
----------------------------------------------------------------